**PDE and Data Generation**

**PDE Discretization FIPY**

In [4]:
# fipy_runner.py

import numpy as np
from fipy import Variable, CellVariable, Grid2D, TransientTerm, DiffusionTerm, ImplicitSourceTerm
from fipy.tools import numerix
import os

def run_simulation(dT0=-0.5, c=0.02, N=6, theta_deg=22.5,
                   steps=1000, save_every=25, save_dir="dataset", run_id="000",
                   seed_radius=0.125, seed_type='circle'):

    dx = dy = 0.025
    nx = ny = 250
    dt = 5e-4
    alpha = 0.015
    tau = 3e-4
    kappa1 = 0.9
    kappa2 = 20.

    mesh = Grid2D(dx=dx, dy=dy, nx=nx, ny=ny)
    phase = CellVariable(name='xi', mesh=mesh, hasOld=True)
    dT = CellVariable(name='DeltaT', mesh=mesh, hasOld=True)

    # Orientation
    theta = np.deg2rad(theta_deg)
    psi = theta + numerix.arctan2(phase.faceGrad[1], phase.faceGrad[0])
    Phi = numerix.tan(N * psi / 2)
    PhiSq = Phi**2
    beta = (1. - PhiSq) / (1. + PhiSq)
    DbetaDpsi = -N * 2 * Phi / (1 + PhiSq)

    Ddia = (1. + c * beta)
    Doff = c * DbetaDpsi
    I0 = Variable(value=((1, 0), (0, 1)))
    I1 = Variable(value=((0, -1), (1, 0)))
    D = alpha**2 * (1. + c * beta) * (Ddia * I0 + Doff * I1)

    heatEq = (TransientTerm() == DiffusionTerm(2.25) + (phase - phase.old) / dt)
    phaseEq = (TransientTerm(tau) == DiffusionTerm(D)
               + ImplicitSourceTerm((phase - 0.5 - kappa1 / numerix.pi * numerix.arctan(kappa2 * dT)) * (1 - phase)))

    # Initial seed
    x, y = mesh.cellCenters
    C = (nx * dx / 2, ny * dy / 2)
    if seed_type == 'circle':
        mask = ((x - C[0])**2 + (y - C[1])**2) < seed_radius**2
    elif seed_type == 'square':
        mask = (np.abs(x - C[0]) < seed_radius) & (np.abs(y - C[1]) < seed_radius)
    else:
        raise ValueError(f"Unsupported seed_type: {seed_type}")
    phase.setValue(1., where=mask)
    dT.setValue(dT0)

    xi_series = []
    dT_series = []

    for i in range(steps):
        phase.updateOld()
        dT.updateOld()
        phaseEq.solve(phase, dt=dt)
        heatEq.solve(dT, dt=dt)

        if i % save_every == 0:
            xi_series.append(phase.value.reshape((ny, nx), order='F').copy())
            dT_series.append(dT.value.reshape((ny, nx), order='F').copy())

    xi_series = np.stack(xi_series)
    dT_series = np.stack(dT_series)

    os.makedirs(save_dir, exist_ok=True)
    np.save(os.path.join(save_dir, f"xi_series_{run_id}.npy"), xi_series)
    np.save(os.path.join(save_dir, f"dT_series_{run_id}.npy"), dT_series)
    if steps%10==0:
        print(steps)
    return xi_series.shape, dT_series.shape


**GENERATE DATA**

In [ ]:
# generate_dataset.py

import os
import json
import itertools
import numpy as np
import time
from PDE_Model import run_simulation  # You must define this in a separate file (e.g., fipy_runner.py)

# === Batch Generation Script ===
def generate_dataset():
    output_dir = "dataset500"
    os.makedirs(output_dir, exist_ok=True)

    param_grid = {
        "dT0": [-0.2, -0.4, -0.6, -0.8],
        "c": [0.005, 0.02, 0.05],
        "N": [4, 6, 8],
        "theta_deg": [0, 15, 30, 45],
        "seed_radius": [0.08, 0.1, 0.15]
        # No "seed_type"
    }


    combinations = list(itertools.product(*param_grid.values()))
    keys = list(param_grid.keys())
    total_runs = len(combinations)

    meta_log = []

    for run_index, combo in enumerate(combinations):
        params = dict(zip(keys, combo))
        run_id = f"{run_index:03d}"

        print(f"\n🔄 Running simulation {run_index + 1}/{total_runs} → ID: {run_id}")
        print(f"    Parameters: {params}")

        start_time = time.time()
        xi_shape, dT_shape = run_simulation(
            **params,
            save_dir=output_dir,
            run_id=run_id
        )
        end_time = time.time()

        print(f"    ✅ Done in {end_time - start_time:.2f} seconds")

        meta_entry = {
            "run_id": run_id,
            "filename_xi": f"xi_series_{run_id}.npy",
            "filename_dT": f"dT_series_{run_id}.npy",
            **params,
            "xi_shape": xi_shape,
            "dT_shape": dT_shape
        }
        meta_log.append(meta_entry)

    with open(os.path.join(output_dir, "metadata.json"), "w") as f:
        json.dump(meta_log, f, indent=2)

    print(f"\n✅ Finished generating {len(meta_log)} simulations. Metadata saved to metadata.json")


if __name__ == "__main__":
    generate_dataset()


**GENERATE DATA MULTIPLE CPUs**

In [ ]:
import os
import json
import itertools
from multiprocessing import Pool
from PDE_Model import run_simulation

# Simulation parameters
param_grid = {
        "dT0": [-0.2, -0.4, -0.6, -0.8],
        "c": [0.005, 0.02, 0.05],
        "N": [4, 6, 8],
        "theta_deg": [0, 15, 30, 45],
        "seed_radius": [0.08, 0.1, 0.15]
        # No "seed_type"
    }
combinations = list(itertools.product(*param_grid.values()))
keys = list(param_grid.keys())

output_dir = "dataset"
os.makedirs(output_dir, exist_ok=True)

def run_wrapper(args):
    run_index, combo = args
    params = dict(zip(keys, combo))
    run_id = f"{run_index:03d}"
    print(f"[{run_id}] Running simulation: {params}")

    xi_shape, dT_shape = run_simulation(
        **params,
        save_dir=output_dir,
        run_id=run_id
    )

    return {
        "run_id": run_id,
        "filename_xi": f"xi_series_{run_id}.npy",
        "filename_dT": f"dT_series_{run_id}.npy",
        **params,
        "xi_shape": xi_shape,
        "dT_shape": dT_shape
    }

def generate_dataset_parallel():
    print(f"🧠 Starting parallel generation on {os.cpu_count()} cores...")
    with Pool() as pool:
        meta_log = pool.map(run_wrapper, enumerate(combinations))

    with open(os.path.join(output_dir, "metadata.json"), "w") as f:
        json.dump(meta_log, f, indent=2)

    print(f"\n✅ Finished generating {len(meta_log)} simulations. Metadata saved to metadata.json")

if __name__ == "__main__":
    generate_dataset_parallel()


**METADATA BUILDER**: Directorty that associates each combinations of parameters with its images

In [ ]:
import os
import json
import itertools

# === Configuration ===
DATASET_DIR = r"C:\Users\Ali\Desktop\798 Project\dataset_500\dataset"
SAVE_DIR = DATASET_DIR

input_keys = ["dT0", "c", "N", "theta_deg", "seed_radius"]
param_grid = {
    "dT0": [-0.2, -0.4, -0.6, -0.8],
    "c": [0.005, 0.02, 0.05],
    "N": [4, 6, 8],
    "theta_deg": [0, 15, 30, 45],
    "seed_radius": [0.08, 0.1, 0.15]
}

# Build full combinations
combinations = list(itertools.product(*param_grid.values()))

# Find available xi_series_XXX.npy files
existing_run_ids = set()
for fname in os.listdir(DATASET_DIR):
    if fname.startswith("xi_series_") and fname.endswith(".npy"):
        run_id = fname.split("_")[-1].split(".")[0]
        existing_run_ids.add(run_id)

# Build metadata ONLY for existing run_ids
metadata = []
for run_id in existing_run_ids:
    run_index = int(run_id)
    if run_index < len(combinations):
        combo = combinations[run_index]
        entry = {
            "run_id": run_id,
            "filename_xi": f"xi_series_{run_id}.npy"
        }
        for key, value in zip(input_keys, combo):
            entry[key] = value
        metadata.append(entry)

# Save metadata.json
metadata_path = os.path.join(SAVE_DIR, "metadata.json")
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Rebuilt metadata.json with {len(metadata)} entries.")


**PLOTTING**: takes number of xi, gives the video

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio.v2 as imageio  # Avoid deprecation warning

# === Settings ===
run_id = "228"
xi_file = r"C:\Users\Ali\Desktop\798 Project\dataset500\xi_series_228.npy" # Fixed here
output_video = f"dendrite_{run_id}.mp4"
fps = 5  # Frames per second
cmap = "plasma"  # Colormap
temp_dir = "temp_frames"

# === Load data ===
xi_series = np.load(xi_file)  # shape: (T, Ny, Nx)

# === Create folder for frames ===
os.makedirs(temp_dir, exist_ok=True)
frame_paths = []

# === Generate frames ===
print("Generating frames...")
for t, frame in enumerate(xi_series):
    fig, ax = plt.subplots(figsize=(5, 5))
    cax = ax.imshow(frame, origin='lower', cmap=cmap)
    ax.set_title(f"Time Step {t}")
    plt.colorbar(cax)

    frame_path = os.path.join(temp_dir, f"frame_{t:03d}.png")
    plt.savefig(frame_path)
    frame_paths.append(frame_path)
    plt.close()

# === Save as MP4 ===
print("Saving MP4 video...")
frames = [imageio.imread(p) for p in frame_paths]
imageio.mimsave(output_video, frames, fps=fps)

# === Clean up ===
for p in frame_paths:
    os.remove(p)
os.rmdir(temp_dir)

print(f"\n✅ Saved video to: {output_video}")


**PLotting2**: Takes parameters and gets the video

In [ ]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import imageio
from PIL import Image

# === Configuration ===
# Set your dataset directory path here
DATASET_DIR = r"C:\Users\Ali\Desktop\798 Project\dataset"

# Choose the parameters to match
target_params = {
    "dT0": -0.6,
    "c": 0.02,
    "N": 6,
    "theta_deg": 30,
    "seed_radius": 0.1
}

# === Load metadata and find matching run ===
metadata_path = os.path.join(DATASET_DIR, "metadata.json")
with open(metadata_path, "r") as f:
    metadata = json.load(f)

def is_close(a, b, tol=1e-6):
    return abs(a - b) < tol if isinstance(a, float) else a == b

match = None
for entry in metadata:
    if all(is_close(entry.get(k), v) for k, v in target_params.items()):
        match = entry
        break

if match is None:
    print("❌ No matching simulation found for the specified parameters.")
else:
    run_id = match["run_id"]
    xi_file = os.path.join(DATASET_DIR, match["filename_xi"])
    print(f"✅ Found match: Run ID {run_id}, loading {xi_file}")

    # === Load data and plot frames ===
    xi_series = np.load(xi_file)
    video_path = os.path.join(DATASET_DIR, f"dendrite_{run_id}.gif")

    frames = []
    for t, frame in enumerate(xi_series):
        fig, ax = plt.subplots(figsize=(8, 8), dpi=150)
        cax = ax.imshow(frame, cmap="plasma", origin="lower", vmin=0, vmax=1, interpolation="bilinear")
        ax.set_title(f"Dendritic Growth: t={t}", fontsize=10)
        fig.colorbar(cax)
        frame_path = os.path.join(DATASET_DIR, f"_temp_frame_{t}.png")
        plt.savefig(frame_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        image = Image.open(frame_path).convert("RGB")
        frames.append(image)

    # Resize all frames to the same shape using PIL
    min_shape = min((img.size for img in frames), key=lambda x: x[0]*x[1])
    frames = [img.resize(min_shape, Image.Resampling.LANCZOS) for img in frames]

    frames[0].save(video_path, save_all=True, append_images=frames[1:], duration=200, loop=0)

    for t in range(len(xi_series)):
        os.remove(os.path.join(DATASET_DIR, f"_temp_frame_{t}.png"))

    print(f"🎥 Video saved to: {video_path}")